In [ ]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

#### **Data Preprocessing**

This provide a sample for loading input training dataset and testing dataset, and randomly select and visualise data samples.

In [ ]:
# Create training dataset direction
dataset = "sample"
if dataset == "sample":
    dataset_dir = "origin_dataset/train_images_directory" #Need to adjust the data path as the training data set

# Read data files and class labels 
def get_dataCategories(dataset_dir):
    import glob

    categories = []
    if dataset == "sample":   #Add more sample functions to allow more input datasets for choosing
        for folder_name in os.listdir(dataset_dir):
            if os.path.isdir(os.path.join(dataset_dir, folder_name)):
                nbr_files = len(
                    glob.glob(os.path.join(dataset_dir, folder_name) + "/*.jpg") #Need to adjust the sample image suffix format to suit the input data
                )
                categories.append(np.array([folder_name, nbr_files]))  
    elif dataset == "sample2":
        for folder_name in os.listdir(dataset_dir):
            if os.path.isdir(os.path.join(dataset_dir, folder_name)):
                nbr_files = len(
                    glob.glob(os.path.join(dataset_dir, folder_name) + "/*.jpg")
                )
                categories.append(np.array([folder_name, nbr_files]))


    categories.sort(key=lambda a: a[0])
    cat = np.array(categories)

    return list(cat[:, 0]), list(cat[:, 1])


categories, nbr_files = get_dataCategories(dataset_dir)

# Create DataFrame
df = pd.DataFrame({"categorie": categories, "numbre of files": nbr_files})
print("number of categories: ", len(categories))
df  # Provide a data frame of your input data

In [ ]:
# Create data tensor
def create_dataset(datadir, categories, img_wid, img_high):
    
    X, y = [], []
    for category in categories:
        path = os.path.join(datadir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                ima_resize_rgb = cv2.resize(img_array, (img_wid, img_high))

                X.append(ima_resize_rgb) 
                y.append(class_num)

            except Exception as e:
                pass

    y = np.array(y)
    X = np.array(X).reshape(y.shape[0], img_wid, img_high, 3) # Resize the data to match the input size requirement, following the direction: sample number, image width, image height, colour channel
    return X, y


img_wid, img_high = 224, 224 # Define the data size

X, y = create_dataset(dataset_dir, categories, img_wid, img_high) 

print(f"X: {X.shape}")
print(f"y: {y.shape}")

In [ ]:
# Randomly visualise training samples 
plt.figure(figsize=(12, 5))
if dataset == 'sample':
    st, end, max_number = 0, 800, 8 # st value defines the start point to select sample
    num = end   # end value defines the end point to select sample
elif dataset == 'sample2':
    st, end, max_number = 0, 800, 8
    num = end

for i in range(max_number): # max_number value defines the number of selected sample
    plt.subplot(2, 4, i + 1)
    idx = np.random.randint(st, end)
    st = end + 1
    end = (i + 2) * num
    # plt.imshow(X[idx][:,:,::-1])
    plt.imshow(X[idx][:, :, ::-1])
    plt.title(f"{i}. {categories[y[idx]]}")
    plt.axis("off")
plt.show()


In [ ]:
# Create testing dataset direction
if dataset == "sample":
    test_dataset_dir = "origin_dataset/test_images_directory"
elif dataset == "sample2":
    test_dataset_dir = "origin_dataset/test_images_directory"


# Converting to scaler format
Y = np.reshape(y, (len(y), 1))

x_train = X
y_train = Y

x_test, y_t = create_dataset(test_dataset_dir, categories, img_wid, img_high)

# One Hot Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_t)

# Verifying the dimension after one hot encoding
print(f"x_train:{x_train.shape},  y_train:{y_train.shape}")
print(f"x_test:{x_test.shape},  y_test:{y_test.shape}")